In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Создание объекта ImageDataGenerator для генерации изображений
data_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Загрузка набора данных Fruits-360 и разделение его на тренировочный и тестовый наборы
train_dataset = data_generator.flow_from_directory('fruits-360/Training', target_size=(224, 224), batch_size=32, subset='training')
test_dataset = data_generator.flow_from_directory('fruits-360/Test', target_size=(224, 224), batch_size=32, subset='validation')

# Создание модели
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(131, activation='softmax')
])

# Компиляция модели
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

# Сохранение модели
model.save('fruit_classifier.h5')

Found 54190 images belonging to 131 classes.
Found 4481 images belonging to 131 classes.
Epoch 1/10
 444/1694 [======>.......................] - ETA: 40:30 - loss: 1.4781 - accuracy: 0.6302

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Загрузка модели
model = tf.keras.models.load_model('fruit_classifier.h5')

# Настройки камеры
cap = cv2.VideoCapture(0)  # Захват видео с первой камеры

# Словарь с названиями фруктов и соответствующими им номерами классов
class_names = {0: 'apple', 1: 'banana', 2: 'orange', 3: 'pear'}

while True:
    # Захват кадра из камеры
    ret, frame = cap.read()

    # Преобразование кадра в изображение, подходящее для входа в модель
    img = cv2.resize(frame, (224, 224))
    img = img.reshape(-1, 224, 224, 3)

    # Прогнозирование класса
    prediction = model.predict(img)[0]
    class_index = tf.argmax(prediction).numpy()
    class_name = class_names[class_index]

    # Обнаружение объектов на кадре
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower = np.array([25, 50, 50])
    upper = np.array([65, 255, 255])
    mask = cv2.inRange(hsv, lower, upper)
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Рисуем контуры объектов на кадре и выводим названия фруктов
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if area > 1000:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, class_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Отображение кадра
    cv2.imshow('Video', frame)

    # Ожидание нажатия клавиши для выхода из программы
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()